In [ ]:
import time
import random
import pathlib
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
chromeDriver = '.\webdriver\driver\chromedriver.exe'

projTypeXPath = '/html/body/center/form/div[2]/label[3]'
typeInXPath = '/html/body/h3[4]/a'

workType = 'pt_proj'
url = f'https://www026190.ccu.edu.tw/{workType}/index.php'
typeInUrl = f'https://www026190.ccu.edu.tw/{workType}/control2.php'
printUrl = f'https://www026190.ccu.edu.tw/{workType}/print_sel.php'
stuUrl = f'https://www026190.ccu.edu.tw/{workType}/frame_stu.php'
stuTitle = '兼任助理、臨時工工作日誌登錄系統'

In [ ]:
######################################################
driver = webdriver.Chrome(chromeDriver)
driver.maximize_window()
driver.get(pathlib.Path('./index.html').resolve().as_uri())
okXPath = '//*[@id="ok"]'

wait = WebDriverWait(driver, 10)

check = True
while check:
    try:
        element = wait.until(
            EC.text_to_be_present_in_element((By.XPATH, okXPath), 'GO')
        )
        print('OK')
        check = False
    except:
        pass

# print(driver.find_element_by_id('mm').get_attribute('value'))
# driver.quit()
# print('END')

In [ ]:
def getFromSingle(selector):
    return driver.find_element_by_css_selector(selector).get_attribute('value')

def getFromMultiple(selector, oneVal = 0):
    ary = []
    for i in driver.find_elements_by_css_selector(selector):
        if i.is_selected():
            ary.append(i.get_attribute('value'))
    if oneVal:
        return ary[0]
    return ary

_workin = getFromMultiple('.workin')
_projID = getFromMultiple('.projID', 1)
_hrs = getFromMultiple('.hrs', 1)
_yy = getFromSingle('#yy')
_mm = getFromSingle('#mm')
_dd = getFromMultiple('.dd')
print(_dd)

In [5]:
driver.get(url)

In [6]:
# 兼任助理
driver.find_element_by_xpath(projTypeXPath).click()
driver.execute_script('javascript:confirmAll()')

In [7]:
# 單一入口登入
# input('PAUSE 1/5')

In [ ]:

check = True
while check:
    try:
        element = wait.until(
            EC.url_contains(stuUrl)
        )
        print('OK')
        check = False
    except:
        pass


In [9]:
def acceptAlert():
    time.sleep(1)
    try:
        anAlert = driver.switch_to.alert
        print(anAlert.text)
        anAlert.accept()
    except:
        print('No Alert')
    ################################
    # try:
    #     wait.until(
    #         EC.alert_is_present(),
    #     )
    #     driver.switch_to.alert.accept()
    # except:
    #     print('No Alert')
    ################################

In [ ]:
acceptAlert()

In [11]:
# 進入輸入工作日誌
driver.get(typeInUrl)

In [12]:
# input('CHECK 2/5')

In [13]:
# 新增工作時段

def exec_a_job(projID, yy, mm, dd, hrs, workin):

    # LOCATING
    driver.switch_to.default_content()
    driver.switch_to.frame('f2')

    # 計畫編號
    element = driver.find_element_by_xpath('//*[@id="accept"]/center/table/tbody/tr[1]/td/select')
    options = element.find_elements_by_tag_name('option')
    for option in options:
        if(option.get_attribute('value') == projID):
            option.click()
            break

    # 日期
    item = driver.find_element_by_name('yy')
    item.clear()
    item.send_keys(yy)
    item = driver.find_element_by_name('mm')
    item.clear()
    item.send_keys(mm)
    item = driver.find_element_by_name('dd')
    item.clear()
    item.send_keys(dd)

    # 時數
    item = driver.find_element_by_name('hrs')
    item.clear()
    item.send_keys(hrs)

    # 內容
    item = driver.find_element_by_name('workin')
    item.clear()
    item.send_keys(workin)

    # SUBMIT
    driver.find_element_by_xpath('//*[@id="accept"]/center/input[1]').click()

In [14]:
# 新增工作時段
for d in _dd:
    exec_a_job(_projID, _yy, _mm, d, _hrs, _workin[random.randint(0, 2)]) # 主計畫

In [15]:
# input('PAUSE 3/5')

In [16]:
# 寫入資料庫
driver.switch_to.default_content()
driver.switch_to.frame('f2')
driver.find_element_by_xpath('//*[@id="accept"]/center/input[2]').click()

In [17]:
#  回主選單
driver.switch_to.default_content()
driver.find_element_by_xpath('/html/body/center[2]/input').click()

In [18]:
# input('PAUSE 4/5')

In [ ]:
acceptAlert()

In [20]:
# 進入產生批號
driver.get(printUrl)

In [21]:

# 計畫編號
element = driver.find_element_by_xpath('/html/body/form/center/table/tbody/tr[1]/td[2]/select')
options = element.find_elements_by_tag_name('option')
for option in options:
    if(option.get_attribute('value') == _projID):
        option.click()
        break

# SUBMIT
driver.find_element_by_xpath('/html/body/form/center/input[1]').click()



In [22]:
# 全選
driver.execute_script("javascript:chkall('form_ck',this)")
driver.find_element_by_xpath('//*[@id="form_ck"]/table/tbody/tr[1]/th[1]/input').click()

# SUBMIT
driver.find_element_by_xpath('//*[@id="form_ck"]/input[1]').click()

In [ ]:
acceptAlert()

In [24]:
# input('PAUSE 5/5')

In [25]:
driver.quit()

In [ ]:
# TODO:
# 1. Alert
# 2. Cancel when listening wait